## Imports & functions

In [2]:
# pip freeze > requirements.txt

In [3]:
import logging
import os
import re
from datetime import date, datetime
from itertools import product

import numpy as np
import pandas as pd
import yaml

from scraping_class import ScrapingClass
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

import requests


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/jules/mediateurenergie/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/jules/mediateurenergie/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1046, in launch_instance
    app.start()
  File "/Users/jules/mediateurenergie/.venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start
    s

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/jules/mediateurenergie/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/jules/mediateurenergie/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1046, in launch_instance
    app.start()
  File "/Users/jules/mediateurenergie/.venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start
    s

AttributeError: _ARRAY_API not found

In [24]:
# HERE = os.path.realpath(os.path.dirname(__file__)) # python script
HERE = os.path.realpath(os.getcwd()) # notebook

class ScrapingEnergieInfo:
    def __init__(self):

        with open(
            os.path.join(
                HERE,
                f"config_files/settings_mediateurenergie_dev.yaml",
            ),
            "r",
        ) as ymlfile:
            self.settings = yaml.safe_load(ymlfile)

        self.today = str(datetime.today().date())
        # try:
        #     self.params = pd.read_csv(f"all_params_{self.today}.csv")
        # except:
        #     print("no params found, starts from scratch")
        #     self.params = pd.DataFrame(columns=['today', 'profile', 'energy_type', 'options'])

        self.sc = ScrapingClass()  # config["driver_path"]
        self.url = self.settings["url"]

        self.options_profile = self.settings["options"]["profile"]
        self.options_energy_type = self.settings["options"]["energy type"]
        self.options_elec_consumption_type = self.settings["options"][
            "elec consumption type"
        ]
        self.options_elec_zip_code = self.settings["options"]["zip code"]["elec"]
        self.options_gas_zip_code = self.settings["options"]["zip code"]["gas"]
        self.options_counter_power = self.settings["options"]["counter power"]
        self.options_gas_consumption = self.settings["options"]["gas consumption"]

        self.all_elec_offers_list = []
        self.all_gas_offers_list = []
        self.all_elec_offers_df = pd.DataFrame()
        self.all_gas_offers_df = pd.DataFrame()
        
        self.current_option = None

    def scrap_comparateur_offre(self, profile, energy_type):
        """
        Running function
        """
        print(f'Scraping profile : {profile} - energy_type : {energy_type}')
        self.profile = profile
        self.energy_type = energy_type
        self.sc.get(self.url)
        self.choose_profile()

        if self.energy_type == "elec":
            options = list(
                product(
                    self.options_elec_consumption_type,
                    self.options_counter_power,
                    self.options_elec_zip_code,
                )
            )
            for ind, option in enumerate(options, start = 1):
                self.current_option = option
                print(f"{profile} - {energy_type} : {ind:02d}/{len(options)} - {option}")
                self.elec_consumption_type = option[0]
                self.counter_power = option[1]
                self.zip_code = option[2]

                self.main_run()
                break

        elif energy_type == "gas":
            options = list(
                product(self.options_gas_consumption, self.options_gas_zip_code)
            )
            for ind, option in enumerate(options, start = 1):
                self.current_option = option
                print(f"{profile} - {energy_type} : {ind:02d}/{len(options)} - {option}")
                self.gas_consumption = option[0]
                self.zip_code = option[1]

                self.main_run()
                break

        # self.cleaning_result()

        #  try:
        #    self.all_offers.to_gbq("team_data.mediateur_national_energie_offers", project_id=settings.GCLOUD_PROJECT, if_exists='append')
        #  except:
        #    pass

        # return self.all_offers

    def choose_profile(self):
        """
        Change the profile of the user (private/professional)
        """
        if self.profile == "professional":
            try:
                self.sc.click_xpath(self.settings["xpath"]["cookie banner"])
            except Exception as e:
                log.info(f"Exception clicking cookies : {e}")
                pass
            sliders = self.sc.find_elements_by_xpath(self.settings["xpath"]["profile slider"])
            for el in sliders:
                try:
                    el.click()
                    break
                except:
                    continue
            
            self.sc.click_css_selector(
                self.settings["selector"]["profile reset button"]
            )
        else:
            pass

    def first_page(self):
        """
        Goes through the first page of the website selecting options from settings file
        """

        try:
            self.sc.click_xpath(self.settings["xpath"]["cookie banner"])
        except Exception as e:
                print(f"Exception clicking cookies : {e}")
                pass
        self.sc.send_xpath(self.settings["xpath"]["zip code"], self.zip_code)
        self.sc.wait(1, 2)
        self.sc.click_xpath(self.settings["xpath"]["city"])
        self.sc.click_xpath(self.settings["xpath"]["energy type"][self.energy_type])

        self.sc.click_xpath(self.settings["xpath"]["next button"])

    def second_page(self):
        """
        Goes through the second page of the website selecting options from settings file
        """
        if self.energy_type == "elec":
            self.sc.click_xpath(self.settings["xpath"]["linky counter"])
            for elmt in ['has contract', 'know prm']:
                try:
                    self.sc.click_xpath(self.settings["xpath"][elmt])
                except:
                    print(f'element not found page 2: {elmt}')

            if self.profile == "private":
                self.sc.click_xpath(
                    self.settings["xpath"]["elec consumption knowPower"]
                )
            self.sc.click_xpath(
                "//select[@id='elec_consumption_power']//option[normalize-space()='"
                + str(self.counter_power)
                + " kVA']"
            )
            self.sc.click_xpath(
                self.settings["xpath"]["elec consumption type"][
                    self.elec_consumption_type
                ]
            )
            self.sc.click_xpath(self.settings["xpath"]["elec consumption knowConso"])
        elif self.energy_type == "gas":
            self.sc.click_xpath(self.settings["xpath"]["gas consumption knowPower"])
            self.sc.send_xpath(
                self.settings["xpath"]["gas consumption"], self.gas_consumption
            )
            self.sc.click_keyboard_enter()

        self.sc.click_css_selector(self.settings["selector"]["submit button"])

    def third_page(self):
        """
        Goes through the third page of the website selecting options from settings file
        """
        self.sc.click_xpath(self.settings["xpath"]["sorted by provider"])
        if self.profile == "professional":
            self.sc.click_xpath(self.settings["xpath"]["displayed price"])
        self.sc.click_css_selector(self.settings["selector"]["submit button"])

    def processing_prices(self, price_string):
        """
        Transforms the string giving the price by extracting the numbers and calculating the matching price
        """
        price = re.search(self.settings["regexp"]["re_price"], price_string)
        if price is None:
            return price_string
        else:
            return (
                float(
                    (
                        price.group(1).replace(",", ".")
                        if not price.group(1) == None
                        else 0
                    )
                ),
                float(
                    (
                        price.group(2).replace(",", ".")
                        if not price.group(2) == None
                        else 0
                    )
                ),
            )

    def processing_consumption_type(self, consumption_type, string):
        """
        Transforms the string giving the consumption type & price by extracting the numbers and calculating the matching date
        """
        if consumption_type == "base":
            price_string = re.search(self.settings["regexp"]["re_base"], string)
        elif consumption_type == "peak":
            price_string = re.search(self.settings["regexp"]["re_hp"], string)
        elif consumption_type == "offpeak":
            price_string = re.search(self.settings["regexp"]["re_hc"], string)
        else:
            price_string = string

        if price_string is None:
            return (None, None)
        elif type(price_string) == str:
            return self.processing_prices(price_string)
        else:
            return self.processing_prices(price_string.group(1))

    def processing_dates(self, date_string):
        """
        Transforms the string giving the date by extracting the numbers and calculating the matching date
        """
        date = re.search(self.settings["regexp"]["re_date"], date_string)
        if date is None:
            return date
        else:
            return datetime.strptime(str(date.group(0)), "%d/%m/%Y")

    def processing_power(self, counter_string):
        """
        Transforms the string giving the counter power by extracting the numbers and calculating the matching date
        """
        counter = re.search(self.settings["regexp"]["re_power"], counter_string)
        if counter is None:
            return counter_string
        else:
            return counter.group(0)

    def additional_columns(self, df):
        """
        Clean columns by splitting prices and converting dates
        """
        df["offer_name"] = df.apply(
            lambda row: (
                row["offer"].split("\n")[0] if pd.notnull(row["offer"]) else None
            ),
            axis=1,
        )
        df["counter_power"] = df.apply(
            lambda row: (
                self.processing_power(row["counter_power"])
                if pd.notnull(row["counter_power"])
                else None
            ),
            axis=1,
        )

        df["subscription_price_ht"] = df.apply(
            lambda row: (self.processing_prices(row["subscription_price"].lower())[0]),
            axis=1,
        )
        df["subscription_price_ttc"] = df.apply(
            lambda row: (self.processing_prices(row["subscription_price"].lower())[1]),
            axis=1,
        )

        df["consumption_price_ht"] = df.apply(
            lambda row: (
                self.processing_consumption_type(
                    row["consumption_type"], row["kwh_price"].lower()
                )[0]
                if pd.notnull(row["kwh_price"])
                else None
            ),
            axis=1,
        )
        df["consumption_price_ttc"] = df.apply(
            lambda row: (
                self.processing_consumption_type(
                    row["consumption_type"], row["kwh_price"].lower()
                )[1]
                if pd.notnull(row["kwh_price"])
                else None
            ),
            axis=1,
        )
        df["hp_consumption_price_ht"] = df.apply(
            lambda row: (
                self.processing_consumption_type("peak", row["kwh_price"].lower())[0]
                if pd.notnull(row["kwh_price"])
                else None
            ),
            axis=1,
        )
        df["hp_consumption_price_ttc"] = df.apply(
            lambda row: (
                self.processing_consumption_type("peak", row["kwh_price"].lower())[1]
                if pd.notnull(row["kwh_price"])
                else None
            ),
            axis=1,
        )
        df["hc_consumption_price_ht"] = df.apply(
            lambda row: (
                self.processing_consumption_type("offpeak", row["kwh_price"].lower())[0]
                if pd.notnull(row["kwh_price"])
                else None
            ),
            axis=1,
        )
        df["hc_consumption_price_ttc"] = df.apply(
            lambda row: (
                self.processing_consumption_type("offpeak", row["kwh_price"].lower())[1]
                if pd.notnull(row["kwh_price"])
                else None
            ),
            axis=1,
        )

        df["begin_validity_date"] = pd.to_datetime(
            df.apply(lambda row: (self.processing_dates(row["begin_date"])), axis=1)
        ).dt.date
        df["end_validity_date"] = pd.to_datetime(
            df.apply(lambda row: (self.processing_dates(row["end_date"])), axis=1)
        ).dt.date
        df["last_updated_date"] = pd.to_datetime(
            df.apply(
                lambda row: (self.processing_dates(str(row["updated_date"]))), axis=1
            )
        ).dt.date

        df["scraping_date"] = date.today()

        return df

    def cleaning_result(self):
        """
        Create the final table that will be uploaded into BigQuery
        """
        if self.all_elec_offers_list:
            self.all_elec_offers_df = pd.DataFrame().from_records(self.all_elec_offers_list)
            self.all_elec_offers_df = self.all_elec_offers_df[
                self.settings["results"]["columns"]
            ]
        if self.all_gas_offers_list:
            self.all_gas_offers_df = pd.DataFrame().from_records(self.all_gas_offers_list)
            self.all_gas_offers_df = self.all_gas_offers_df[
                self.settings["results"]["columns"]
            ]
        print(self.all_gas_offers_df)
        print(self.all_elec_offers_df)

        if not any([type(self.all_elec_offers_df) == pd.DataFrame, type(self.all_gas_offers_df) == pd.DataFrame]):
            print("Both list are empty")
            self.all_offers = None
            return

        if (
            not self.all_elec_offers_df.empty
            and self.all_gas_offers_df.empty
            # type(self.all_elec_offers_df) == pd.DataFrame
        ):
            self.all_offers = self.all_elec_offers_df
        elif (
            # type(self.all_gas_offers) == pd.DataFrame
            # and type(self.all_elec_offers) == list
            self.all_elec_offers_df.empty
            and not self.all_gas_offers_df.empty
        ):
            self.all_offers = self.all_gas_offers_df
        else:
            self.all_offers = pd.concat(
                [self.all_elec_offers_df, self.all_gas_offers_df], ignore_index=True
            )

        self.all_offers.rename(columns=self.settings["results"]["rename"], inplace=True)
        self.all_offers = self.additional_columns(df=self.all_offers)
        self.all_offers = self.all_offers.loc[:, ~self.all_offers.columns.duplicated()]

        self.all_offers = self.all_offers[self.settings["results"]["ordered_columns"]]

        try:
            self.all_offers["gas_zone"].fillna(value=np.nan, inplace=True)
            self.all_offers["gas_zone"] = self.all_offers["gas_zone"].astype("int64")
        except:
            pass
        self.all_offers["hp_consumption_price_ht"] = self.all_offers[
            "hp_consumption_price_ht"
        ].astype("float64")
        self.all_offers["hp_consumption_price_ttc"] = self.all_offers[
            "hp_consumption_price_ttc"
        ].astype("float64")
        self.all_offers["hc_consumption_price_ht"] = self.all_offers[
            "hc_consumption_price_ht"
        ].astype("float64")
        self.all_offers["hc_consumption_price_ttc"] = self.all_offers[
            "hc_consumption_price_ttc"
        ].astype("float64")

        for date_col in self.all_offers.columns:
            if 'date' in date_col:
                self.all_offers[date_col] = self.all_offers[date_col].astype(str)
  

    def main_run(self):
        """
        Main running function for a unique combination of options
        """
        # param_checked, params = self.check_params()
        # if param_checked:
        #     self.new_simulation()
        #     print(f"params already used ({' - '.join([param for param in params])})")
        #     return
        
        try:
            self.first_page()
            self.second_page()
            self.third_page()
        except Exception as e:
            print(f"Exception in main_run : {e}")
            pass

## Init scraper

In [26]:
scraper = ScrapingEnergieInfo()

In [27]:
scraper.scrap_comparateur_offre(profile='private',energy_type='elec')

Scraping profile : private - energy_type : elec
private - elec : 01/2 - ('base', 9, 13008)


## Tests

In [28]:
# Collect cookies
def get_cookies(driver):
    cookies = {}
    for cookie in driver.get_cookies():
        cookies.update({cookie['name']:cookie['value']})
    return cookies

# Retrieve token
def get_token_search(html):
    soup = BeautifulSoup(html, 'html.parser')
    token_elmt = soup.find(id='offer_filters__token')
    token_filter = token_elmt.get('value')
    return token_filter

In [31]:
# Get offer ids
cookies = get_cookies(scraper.sc.driver)
token_filter = get_token_search(scraper.sc.driver.page_source)
last = False
ind = 1

offers_ids = []

headers = {
    'accept': '*/*',
    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    'priority': 'u=1, i',
    'referer': 'https://comparateur-offres.energie-info.fr/',
    'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}

params = {
    's': '',
    'offer_filters[fullGreen]': '0',
    'offer_filters[fullOnLine]': '0',
    'offer_filters[period]': '12',
    'offer_filters[linky]': '0',
    'offer_filters[sortBy]': '3',
    'offer_filters[sortDirection]': 'ASC',
    'offer_filters[_token]': token_filter,
}

while last is False:
    url_filtered_results = f'https://comparateur-offres.energie-info.fr/results/{ind}'
    print(url_filtered_results)
    response = requests.get(url_filtered_results, params=params, cookies=cookies, headers=headers)
    

    soup_test = BeautifulSoup(response.json().get('html'), 'html.parser')
    offers = soup_test.find_all('div', class_ ='offre offer')
    offers_ids.extend([offer.get('data-id') for offer in offers])
    
    if response.json().get('lastPage'):
        last = True

    ind+=1

print(f'offers : {len(offers_ids)}')

https://comparateur-offres.energie-info.fr/results/1
https://comparateur-offres.energie-info.fr/results/2
https://comparateur-offres.energie-info.fr/results/3
https://comparateur-offres.energie-info.fr/results/4
offers : 60


In [49]:
scraper.elec_consumption_type

'base'

In [32]:
# Update cookies
cookies = get_cookies(scraper.sc.driver)
url_detail = 'https://comparateur-offres.energie-info.fr/detail/'

offers_soup = []

for ind,id in enumerate(offers_ids, start = 1):
    url_id = url_detail+id
    print(f'{ind:02d} - {url_id}')
    res = requests.get(url = url_id, cookies=cookies, headers=headers)
    offers_soup.append({'id':id, 'soup':BeautifulSoup(res.content, 'html.parser')})

01 - https://comparateur-offres.energie-info.fr/detail/12419
02 - https://comparateur-offres.energie-info.fr/detail/12984
03 - https://comparateur-offres.energie-info.fr/detail/13676
04 - https://comparateur-offres.energie-info.fr/detail/13303
05 - https://comparateur-offres.energie-info.fr/detail/13903
06 - https://comparateur-offres.energie-info.fr/detail/13883
07 - https://comparateur-offres.energie-info.fr/detail/13681
08 - https://comparateur-offres.energie-info.fr/detail/13686
09 - https://comparateur-offres.energie-info.fr/detail/13690
10 - https://comparateur-offres.energie-info.fr/detail/13684
11 - https://comparateur-offres.energie-info.fr/detail/14081
12 - https://comparateur-offres.energie-info.fr/detail/6887
13 - https://comparateur-offres.energie-info.fr/detail/13689
14 - https://comparateur-offres.energie-info.fr/detail/13688
15 - https://comparateur-offres.energie-info.fr/detail/6539
16 - https://comparateur-offres.energie-info.fr/detail/6534
17 - https://comparateur-of

In [33]:
offers_soup[0].keys()

dict_keys(['id', 'soup'])

In [35]:
def get_offer_dict_raw(offer_soup):
    offer = {}    
    result_rows = offer_soup.find(id = 'electricite').find_all('tr')
    
    for row in result_rows:
    ## Add additionnal keys from rows in result based on yaml settings for results columns
        children = row.children

        if any([child.name == "td" for child in children]): # Filter rows that have no values
            
            key = row.find('th').get_text(strip = True)
            value = row.find('td').get_text(strip = True)

            if key in scraper.settings['results']['columns']: # Only keeps desired columns
                offer[key] = value
    return offer

In [50]:
# elec offers
for offer in offers_soup:
    offer_dict = {'offer_id' : offer['id'],
            #  "provider_name" : provider,
             'profile' : scraper.profile,
             'energy_type' : scraper.energy_type,
             'consumption_type' : scraper.elec_consumption_type,
             'gas_category' : None,
             'gas_zone' : None,
             }

    offer_dict.update(get_offer_dict_raw(offer['soup']))

    offer.update({'dict':offer_dict})

In [51]:
df = pd.DataFrame().from_records([offer.get('dict') for offer in offers_soup])

In [52]:
df.rename(columns=scraper.settings["results"]["rename"], inplace=True)

In [53]:
scraper.additional_columns(df=df)

,offer_id,profile,energy_type,consumption_type,gas_category,gas_zone,offer,price_comment,subscription_price,kwh_price,...,consumption_price_ht,consumption_price_ttc,hp_consumption_price_ht,hp_consumption_price_ttc,hc_consumption_price_ht,hc_consumption_price_ttc,begin_validity_date,end_validity_date,last_updated_date,scraping_date
0,12419,private,elec,base,None,None,OFFRE ELECTRICITE PROMOType : Base,,"154,08€ HT /\n ...",Base :\n ...,...,0.1604,0.0,None,None,NaN,NaN,2023-08-01,NaT,2024-04-12,2024-07-15
1,12984,private,elec,base,None,None,OFFRE PRIX FIXE 100% VERTEType : Base,,"154,08€ HT /\n ...",Base :\n ...,...,0.1541,0.0,None,None,NaN,NaN,2023-08-01,NaT,2024-04-12,2024-07-15
2,13676,private,elec,base,None,None,OFFRE PRIX FIXE -21% ALPIQType : Base,,"154,08€ HT /\n ...",Base :\n ...,...,0.1491,0.0,None,None,NaN,NaN,2023-08-01,NaT,2024-04-12,2024-07-15
3,13303,private,elec,base,None,None,Électricité verte 100 % locale - Prix fixe 1 a...,,"169,49€ HT /\n ...",Base :\n ...,...,0.1472,0.0,None,None,NaN,NaN,2023-04-17,NaT,2024-05-06,2024-07-15
4,13903,private,elec,base,None,None,Électricité verte 100 % française - Prix fixe ...,,"169,49€ HT /\n ...",Base :\n ...,...,0.1396,0.0,None,None,NaN,NaN,2024-05-06,NaT,2024-05-06,2024-07-15
5,13883,private,elec,base,None,None,Mon contrat ElectricitéType : Base,Le prix du kWh (HTT en €) intègre l'acheminement.,"152,18€ HT /\n ...",Base :\n ...,...,0.1537,0.0,None,None,NaN,NaN,2024-05-01,NaT,2024-05-07,2024-07-15
6,13681,private,elec,base,None,None,Vert ÉlectriqueType : Base,,"154,08€ HT /\n ...",Base :\n ...,...,0.1809,0.0,None,None,NaN,NaN,2024-03-19,NaT,2024-03-31,2024-07-15
7,13686,private,elec,base,None,None,Vert Électrique Week-EndType : Base,,"154,08€ HT /\n ...",Heures Semaine :\n ...,...,NaN,NaN,None,None,NaN,NaN,2024-03-19,NaT,2024-06-11,2024-07-15
8,13690,private,elec,base,None,None,Zen OnlineType : Base,,"154,08€ HT /\n ...",Base :\n ...,...,0.1696,0.0,None,None,NaN,NaN,2024-03-19,NaT,2024-05-15,2024-07-15
9,13684,private,elec,base,None,None,Vert Électrique RégionalType : Base,,"154,08€ HT /\n ...",Base :\n ...,...,0.1863,0.0,None,None,NaN,NaN,2024-03-19,NaT,2024-03-31,2024-07-15


---

## Restart

In [25]:
scraper.sc.quit()